## 1. Question Formulation  

Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful.

The goal of this analysis is to predict click-through rates for online advertising for Criteo by generating individual click-through predictions for different user/ad/context combinations. Criteo is an advertising technology company that offers a variety of solutions for helping e-commerce companies understand and serve targeted ads to their customers. 

The formula for click-through rate is (Total Clicks on Ad) / (Total Times Users Saw the Add). Click through rates are useful to companies in several ways:
1. E-commerce sites or advertising companies could compare the click rate of one ad to another to determine which one is doing better, and use that information to make adjustments to what ads they are serving. 
2. E-commerce sites could use click-through rates as part of their projections for web traffic and sales.
3. Platforms that serve ads, like Google, have a vested interest in serving users ads that are relevant to them, so they will offer discounts to advertisers on ads that have a high click through rate, as well as other desirable qualities.

Criteo formulated the question this way for the Kaggle competition using this data set: "Given a user and the page he is visiting, what is the probability that he will click on a given ad?"  By predicting probability of click for particular user/ad/context combinations, we aren't directly predicting click through rate, but rather predicting the elements we will use to calculate click through rate. By making predictions on the test set, and taking the N prediction rows that correspond to a particular prediction, and the C positive examples where our model predicts that the user would click, we can say, "if we showed users this ad N times, we predict that the click through rate would be C/N."

The train and test data sets for this problem have 40 columns:
- a label column, with a 1 or 0 indicating that the user did or did not click through the ad, respectively
- 13 columns with numerical data
- 26 columns with categorical data, where all categorical values are hexified for confidentiality
Since we have no column labels and no insight into the categories in the categorical feature columns, we are limited in the number of questions we can answer. We can't calculate and compare the click-through rates of different ads or different ad contexts, or compare the click-through rates for different users, without knowing which columns refer to users, ads, or different elements of context. 

As a result, the only click-through rate we can calculate is one for the entire data set. We can make a prediction for each individual row of data (representing a user, ad, and context), and then we can calculate a click-through rate for the entire set of predictions. Without knowing anything about the data (is it all for one advertiser? for one advertising campaign? for ads displayed on a single website?), it's hard to know what question the click through rate will help us answer. We could answer "what is the baseline click-through rate for this set of data," or perhaps, if we explore feature importance in our final model, "which features are most useful for predicting click-through rate on this data"?

TODO: clarify what question we're answering above, and introduce what algorithm we will use

## 2. Algorithm Explanation 

Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithm that you will perform.

In [ ]:
# answer #2 in this section
# STEVE

## 3. EDA & Discussion of Challenges  

Determine 2-3 relevant EDA tasks that will help you make decisions about how you implement the algorithm to be scalable. Discuss any challenges that you anticipate based on the EDA you perform.

In [ ]:
# answer #3 in this section
# Alex
#   - explore log transforming a selected set of features (Saturday)

## 4. Algorithm Implementation  

Develop a ‘homegrown’ implementation of the algorithm, apply it to the training dataset and evaluate your results on the test set.

In [ ]:
# answer #4 in this section

# 4a Feature Engineering
# Emily
#   - Run Random Forest for enhanced feature selection; may lead to new "final pre-processed" dataset (complete by Saturday night)
# Alex
#   - Try weighted LR (due to imbalanced label distribution)

# 4b Homegrown Factorization Machine and Logistic Regression
# Divya --> homegrown algorithms, explaining which parts are logistic vs FM 

# 4c Logistic Regression Implementation in Spark ML
# Steve/Alex --> actually running logistic regression ML, enhancements: feature importance, loss plots [Alex to run on the cluster]
#   - Complete hyperparameter tuning with sample data on "final pre-processed" dataset (Saturday/Sunday)
#   - Using tuned hyperameters, run LR with full "final pre-processed" dataset (Saturday/Sunday)

## 5. Application of Course Concepts 

Pick 3-5 key course concepts and discuss how your work on this assignment illustrates an understanding of these concepts.

In [ ]:
# answer #5 in this section
# Emily 